<h2 id="50-データの入手整形">50. データの入手・整形</h2>
<p><a href="https://archive.ics.uci.edu/ml/datasets/News+Aggregator">News Aggregator Data Set</a>をダウンロードし、以下の要領で学習データ（<code class="language-plaintext highlighter-rouge">train.txt</code>），検証データ（<code class="language-plaintext highlighter-rouge">valid.txt</code>），評価データ（<code class="language-plaintext highlighter-rouge">test.txt</code>）を作成せよ．</p>
<ol>
<li>ダウンロードしたzipファイルを解凍し，<code class="language-plaintext highlighter-rouge">readme.txt</code>の説明を読む．</li>
<li>情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．</li>
<li>抽出された事例をランダムに並び替える．</li>
<li>抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれ<code class="language-plaintext highlighter-rouge">train.txt</code>，<code class="language-plaintext highlighter-rouge">valid.txt</code>，<code class="language-plaintext highlighter-rouge">test.txt</code>というファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．</li>
</ol>
<p>学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．</p>


In [359]:
!unzip -y data/NewsAggregatorDataset.zip

UnZip 6.00 of 20 April 2009, by Info-ZIP.  Maintained by C. Spieler.  Send
bug reports using http://www.info-zip.org/zip-bug.html; see README for details.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files

In [360]:
import pandas as pd
# csvファイルを読み込む
df = pd.read_table('data/newsCorpora.csv', header=None, sep='\\t', engine='python')
df.columns = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
# 先頭5つ表示
df.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [361]:
# PUBLISHERが特定の行のみを取り出す
publishers = ['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']
daily_mails = df[df['PUBLISHER'].isin(publishers)]
len(daily_mails)  # 取り出した行数

13356

In [362]:
from sklearn.model_selection import train_test_split
# 訓練データ、検証データ、テストデータに分ける
train_data, non_train, train_target, non_train_target = train_test_split(daily_mails[['TITLE', 'CATEGORY']], daily_mails['CATEGORY'], train_size=0.8, random_state=0)
valid_data, test_data = train_test_split(non_train, train_size=0.5, random_state=0)
print(len(train_data), len(valid_data), len(test_data))

# テキストファイルに書き込む
train_data.to_csv('work/train.txt', header=None, index=None, sep='\t')
valid_data.to_csv('work/valid.txt', header=None, index=None, sep='\t')
test_data.to_csv('work/test.txt', header=None, index=None, sep='\t')

10684 1336 1336


In [363]:
train_data.value_counts('CATEGORY')

CATEGORY
b    4503
e    4254
t    1210
m     717
dtype: int64

In [364]:
valid_data.value_counts('CATEGORY')

CATEGORY
b    559
e    522
t    152
m    103
dtype: int64

In [365]:
test_data.value_counts('CATEGORY')

CATEGORY
b    565
e    518
t    163
m     90
dtype: int64

<h2 id="51-特徴量抽出">51. 特徴量抽出</h2>
<p>学習データ，検証データ，評価データから特徴量を抽出し，それぞれ<code class="language-plaintext highlighter-rouge">train.feature.txt</code>，<code class="language-plaintext highlighter-rouge">valid.feature.txt</code>，<code class="language-plaintext highlighter-rouge">test.feature.txt</code>というファイル名で保存せよ．
なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．</p>


In [366]:
# 出現する単語の個数をカウントしてベクトル化する
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle

# 特徴ベクトル
vec_count = CountVectorizer()
vec_count.fit(train_data['TITLE'])  # 語彙の生成
X_train = vec_count.transform(train_data['TITLE'])  # 特徴ベクトルの生成
y_train = train_target.map({'b': 0, 'e': 1, 't': 2, 'm': 3})  # 正解カテゴリ
X_valid = vec_count.transform(valid_data['TITLE'])
X_test = vec_count.transform(test_data['TITLE'])


# ファイルへの書き込み
with (open('work/train.feature.pkl', 'wb') as train_text,
      open('work/valid.feature.pkl', 'wb') as valid_text,
      open('work/test.feature.pkl', 'wb') as test_text):
    pickle.dump(X_train, train_text)
    pickle.dump(X_valid, valid_text)
    pickle.dump(X_test, test_text)

# 表示
print(f'語彙数：{len(vec_count.vocabulary_)}')
print(vec_count.vocabulary_)
print(sorted(vec_count.vocabulary_.items(), key=lambda x: x[1], reverse=True))
print(f'X_train shape : {X_train.shape}')

# 書き込めているか確認
np.set_printoptions(threshold=10)
with open('work/test.feature.pkl', 'rb') as f:
    data = pickle.load(f)
    print(f'X_test shape  : {data.shape}')
    print(data.toarray())

語彙数：12858
{'truecar': 11878, 'sputters': 10824, 'onto': 8046, 'wall': 12400, 'street': 11024, 'amid': 761, 'crowd': 2998, 'of': 7983, 'car': 2066, 'shopping': 10302, 'sites': 10433, 'update': 12112, 'ecb': 3860, 'says': 9979, 'further': 4870, 'euro': 4146, 'strengthening': 11028, 'would': 12724, 'trigger': 11841, 'looser': 6892, 'monetary': 7524, 'are': 945, 'asia': 1012, 'crude': 3007, 'oil': 8006, 'buyers': 1963, 'too': 11678, 'relaxed': 9435, 'over': 8176, 'iraq': 6111, 'clyde': 2498, 'russell': 9862, 'madonna': 6995, 'reveals': 9617, 'little': 6826, 'much': 7631, 'as': 1002, 'she': 10248, 'poses': 8725, 'in': 5866, 'fishnet': 4577, 'tights': 11616, 'and': 785, 'thong': 11554, 'for': 4709, 'max': 7174, 'mosley': 7578, 'chilling': 2324, 'ruling': 9840, 'that': 11505, 'anyone': 869, 'can': 2020, 'airbrush': 612, 'their': 11515, 'history': 5572, 'obamacare': 7940, 'shuffles': 10339, 'health': 5446, 'plan': 8582, 'market': 7101, 'share': 10233, 'report': 9503, 'finds': 4539, 'frankie': 

In [367]:
# TF-IDF法
from sklearn.feature_extraction.text import TfidfVectorizer

vec_tfidf = TfidfVectorizer(stop_words='english')
X2_train = vec_tfidf.fit_transform(train_data['TITLE'])
print(X2_train.toarray())
print(X2_train.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(10684, 12603)


<h2 id="52-学習">52. 学習</h2>
<p>51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．</p>


In [368]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=100)  # モデルを指定
result = lr.fit(X_train, y_train)  # 訓練データで学習をする
print(f'train accuracy : {result.score(X_train, y_train)}')  # 正解率

train accuracy : 0.9956008985398727


/Users/nyuton/.pyenv/versions/anaconda3-2022.10/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<h2 id="53-予測">53. 予測</h2>
<p>52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．</p>


In [375]:
pred = lr.predict(X_train)  # 訓練データを用いてカテゴリ予測
probabilitys = lr.predict_proba(X_train)  # それぞれのデータについて、カテゴリの確率分布を求める
# 表示
print(f'訓練データの正解率 : {lr.score(X_train, y_train)}')
print(f'予想カテゴリとその大きさ : {pred.shape}')
print(pred)
print(f'確率分布とその大きさ : {probabilitys.shape}')
print(probabilitys)

訓練データの正解率 : 0.9956008985398727
予想カテゴリとその大きさ : (10684,)
[0 0 0 ... 0 0 1]
確率分布とその大きさ : (10684, 4)
[[9.45246937e-01 3.84353092e-02 9.17158565e-03 7.14616857e-03]
 [9.98325006e-01 2.77359280e-04 1.13734326e-03 2.60291484e-04]
 [9.62203273e-01 2.66654539e-02 2.89614947e-03 8.23512312e-03]
 ...
 [9.64238679e-01 1.77011882e-02 1.22602690e-02 5.79986333e-03]
 [9.76613806e-01 4.18522048e-03 6.95330948e-03 1.22476639e-02]
 [1.24892083e-03 9.97430334e-01 7.88251805e-04 5.32493255e-04]]


<h2 id="54-正解率の計測">54. 正解率の計測</h2>
<p>52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ．</p>


In [370]:
# 評価データと検証データのカテゴリを作成
y_test = test_data['CATEGORY'].map({'b': 0, 'e': 1, 't': 2, 'm': 3})
y_valid = valid_data['CATEGORY'].map({'b': 0, 'e': 1, 't': 2, 'm': 3})
# 訓練データ、評価データ、検証データの正答率
print(f'train accuracy : {lr.score(X_train, y_train)}')
print(f'test  accuracy : {lr.score(X_test, y_test)}')
print(f'valid accuracy : {lr.score(X_valid, y_valid)}')

train accuracy : 0.9956008985398727
test  accuracy : 0.9101796407185628
valid accuracy : 0.9206586826347305


In [377]:
from sklearn.metrics import accuracy_score
# 真の正解カテゴリy_trainと予測カテゴリpred_yの一致している割合から正解率を求める
# 訓練データ
pred_train = lr.predict(X_train)
print("train accuracy : ", accuracy_score(y_train, pred_train))
# 評価データ
pred_test = lr.predict(X_test)
print("test accuracy : ", accuracy_score(y_test, pred_test))

train accuracy :  0.9956008985398727
test accuracy :  0.9101796407185628


<h2 id="55-混同行列の作成">55. 混同行列の作成</h2>
<p>52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．</p>


<h2 id="56-適合率再現率f1スコアの計測">56. 適合率，再現率，F1スコアの計測</h2>
<p>52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．</p>


<h2 id="57-特徴量の重みの確認">57. 特徴量の重みの確認</h2>
<p>52で学習したロジスティック回帰モデルの中で，重みの高い特徴量トップ10と，重みの低い特徴量トップ10を確認せよ．</p>


<h2 id="58-正則化パラメータの変更">58. 正則化パラメータの変更</h2>
<p>ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，学習時の過学習（overfitting）の度合いを制御できる．異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．</p>


<h2 id="59-ハイパーパラメータの探索">59. ハイパーパラメータの探索</h2>
<p>学習アルゴリズムや学習パラメータを変えながら，カテゴリ分類モデルを学習せよ．検証データ上の正解率が最も高くなる学習アルゴリズム・パラメータを求めよ．また，その学習アルゴリズム・パラメータを用いたときの評価データ上の正解率を求めよ．</p>
